In [ ]:

import os
import pickle
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Create Dataframe

In [ ]:
# Define paths to your training and testing directories
# data_dir = 'Datasets/combined_dataset_processed'
data_dir = 'Datasets/combined_dataset_enhanced_npy_6_channels'

# Function to add images from a directory to a list
def process_directory(directory, data_list):
    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)

        # Check if it's a directory
        if os.path.isdir(class_dir):
            # Loop through each image in the folder
            for image_name in os.listdir(class_dir):
                if image_name.endswith('.npy'):  # Ensure the file is a .npy file
                    image_path = os.path.join(class_dir, image_name)
                    image = np.load(image_path)

                    # Check if the image is of size 128x128 with 6 channels
                    if image.shape == (128, 128, 6):
                        # Append to the data list
                        data_list.append({'filepath': image_path, 'label': class_name})

# Initialize an empty list for storing data
data_list = []
# Add training images to the data list
process_directory(data_dir, data_list)
# Create a DataFrame from the list
df = pd.DataFrame(data_list)
# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)
# Print the amount of images per category before balancing
print("Images per category before balancing:")
print(df['label'].value_counts())

# Split the data into training, validation, and test sets
train_df, temp_test_val_df = train_test_split(df, train_size=0.7, random_state=42)
test_df, val_df = train_test_split(temp_test_val_df, test_size=0.5, random_state=42)

# Balance the test and validation sets
# Determine the smallest class size within each of the test and validation sets
min_test_class_size = test_df['label'].value_counts().min()
min_val_class_size = val_df['label'].value_counts().min()

# Determine the smallest size between the two for a uniform approach
uniform_min_size = min(min_test_class_size, min_val_class_size)

# Function to reduce class sizes
def balance_classes(df, target_size):
    balanced_df = pd.DataFrame()  # Initialize an empty DataFrame to hold the balanced data
    for label in df['label'].unique():
        subset = df[df['label'] == label].sample(n=target_size, random_state=42)
        balanced_df = pd.concat([balanced_df, subset])
    return balanced_df

# Apply balancing
test_df = balance_classes(test_df, uniform_min_size)
val_df = balance_classes(val_df, uniform_min_size)

# Calculate and print split ratios
total_samples = len(df)
train_ratio = len(train_df) / total_samples
val_ratio = len(val_df) / total_samples
test_ratio = len(test_df) / total_samples

print(f"\nTotal samples: {total_samples}")
print(f"Training set: {train_ratio:.2f} ({len(train_df)} samples)")
print(f"Validation set: {val_ratio:.2f} ({len(val_df)} samples)")
print(f"Test set: {test_ratio:.2f} ({len(test_df)} samples)\n")

# Preprocessing

In [ ]:
# Set the image size and batch size
image_size = (128, 128)
batch_size = 64 

# Encode the labels to integers
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])
val_df['encoded_label'] = label_encoder.transform(val_df['label'])
test_df['encoded_label'] = label_encoder.transform(test_df['label'])

def set_shapes(img, label, img_shape=(128, 128, 6), label_shape=(7, )):
    img.set_shape(img_shape)
    label.set_shape(label_shape)
    return img, label

def preprocess_npy(file_path, label):
    # Load .npy file
    image = np.load(file_path.numpy())
    # Normalize image data
    image = image.astype(np.float32) / 255.0
    # One-hot encode the label
    label = tf.one_hot(label, depth=7)  # 7 classes
    return image, label

def create_dataset(df, batch_size):
    file_paths = df['filepath'].values
    labels = df['encoded_label'].values
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.map(lambda x, y: tf.py_function(preprocess_npy, [x, y], [tf.float32, tf.float32]), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(set_shapes)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
    return dataset

# Create datasets
train_dataset = create_dataset(train_df, batch_size)
val_dataset = create_dataset(val_df, batch_size)
test_dataset = create_dataset(test_df, batch_size)

# Calculate class weights for the training data
labels_for_class_weight = train_df['encoded_label'].values
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(labels_for_class_weight),
                                     y=labels_for_class_weight)

# Convert class weights to a dictionary to pass to model.fit
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}
print(class_weight_dict)

# for images, labels in train_dataset.take(1):  # Take a single batch from the dataset
#     print("Image batch shape:", images.shape)
#     print("Label batch shape:", labels.shape)

# Model Definition

In [ ]:
input_shape = (128, 128, 6)
l2_reg = 0.001  # Regularization strength
dropout = [0.25, 0.4]
n_filters = [128, 64, 32, 128]

model = Sequential([
    Input(shape=input_shape),
    # First Conv Block
    Conv2D(n_filters[0], (3, 3), padding='same', activation='relu', kernel_regularizer=l2(l2_reg)), 
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(dropout[0]),
    # Second Conv Block
    Conv2D(n_filters[1], (3, 3), padding='same', activation='relu', kernel_regularizer=l2(l2_reg)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(dropout[0]),
    # Third Conv Block
    Conv2D(n_filters[2], (3, 3), padding='same', activation='relu', kernel_regularizer=l2(l2_reg)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(dropout[0]),
    # Fourth Conv Block
    Conv2D(n_filters[1], (3, 3), padding='same', activation='relu', kernel_regularizer=l2(l2_reg)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(dropout[0]),
    # Flatten and Dense Layers
    Flatten(),
    Dense(n_filters[3], activation='relu', kernel_regularizer=l2(l2_reg)),
    BatchNormalization(),
    Dropout(dropout[1]),
    Dense(7, activation='softmax', kernel_regularizer=l2(l2_reg))  # 7 classes for the output
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Callbacks

In [ ]:
# Early stopping to prevent overfitting. This stops training when the model's performance on the validation set starts to degrade.
early_stopper = EarlyStopping(
    monitor='val_loss',  # Metric to be monitored
    patience=3,         # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored metric
)

# ModelCheckpoint callback
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
checkpoint = ModelCheckpoint(
    f'logs/model_checkpoint_{timestamp}.keras',  # Path where to save the model
    monitor='val_loss',     # Metric to monitor
    save_best_only=False,    # Save only the best model. Set False to save the model at the end of every epoch so restarting from specific epoch is possible
    save_weights_only=False, # Save only the weights
    mode='min',             # Minimize the monitored metric (val_loss) min before
    verbose=1               # Verbose output
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=2,
    min_lr=0.0001,
    cooldown=3,
    verbose=1
)

# Model Training

In [ ]:
# Load the last saved weights
# model.load_weights('logs/model_checkpoint_20240225_085054.keras')

epochs = 50  # When resuming training, set epochs to the total number of epochs you want to train, not just the additional epochs. The model.fit() method continues training for the specified number of epochs, starting from the current epoch count.

steps_per_epoch = np.ceil(len(train_df) / batch_size).astype(int)
validation_steps = np.ceil(len(val_df) / batch_size).astype(int)


history = model.fit(
    train_dataset,  # Training data
    epochs=epochs,  # Number of epochs
    validation_data=val_dataset,  # Validation data
    class_weight=class_weight_dict,  # Class weights
    steps_per_epoch=len(train_df) // batch_size,  # Number of steps per epoch
    validation_steps=len(val_df) // batch_size,  # Number of validation steps
    verbose=1,  # Show training log
    callbacks=[early_stopper, checkpoint, reduce_lr]
)

# Save the training history for later analysis
with open(f'logs/training_history_{timestamp}.pkl', 'wb') as file:
    pickle.dump(history.history, file)

# Evaluation and Visualization

In [ ]:
test_steps = np.ceil(len(test_df) / batch_size).astype(int)

# Ensure test_dataset is correctly prepared with batching and without shuffling for evaluation
test_loss, test_accuracy = model.evaluate(test_dataset, steps=test_steps)
print("Test accuracy: ", test_accuracy)

# Predictions on the test set
predictions = model.predict(test_dataset, steps=test_steps)
predicted_classes = np.argmax(predictions, axis=1)  # because of softmax output from model

# If true labels are one-hot encoded, convert them to integer labels
true_labels_integer = np.concatenate([y.numpy() for x, y in test_dataset.unbatch().batch(batch_size).take(test_steps)], axis=0)
if true_labels_integer.ndim > 1:  # Checking if labels are one-hot encoded
    true_labels_integer = np.argmax(true_labels_integer, axis=1)

# Ensure true_labels and predicted_classes arrays are aligned in length
true_labels_integer = true_labels_integer[:len(predicted_classes)]

# Assuming 'label_encoder' is defined and used for encoding labels
class_labels = label_encoder.inverse_transform(range(len(label_encoder.classes_)))

# Generate classification report
print("Classification Report:")
print(classification_report(true_labels_integer, predicted_classes, target_names=class_labels))

# Confusion Matrix
cm = confusion_matrix(true_labels_integer, predicted_classes)
plt.figure(figsize=(10, 10))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=45)
plt.yticks(tick_marks, class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

Conducting error analysis
This can be done by examining misclassified examples, which can provide insights into what types of errors the model is making

In [ ]:
with open(f'logs/training_history_20240301_170452.pkl', 'rb') as file:
    history = pickle.load(file)
# Learning Curves
# Plot training & validation accuracy values
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Precsion-Recall Curve
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import label_binarize

# Binarize the labels for multi-class
y_bin = label_binarize(true_labels_integer, classes=np.arange(len(class_labels)))
n_classes = y_bin.shape[1]

# Compute precision-recall curve for each class
precision = dict()
recall = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_bin[:, i], predictions[:, i])

# Plot the precision-recall curve
for i in range(n_classes):
    plt.plot(recall[i], precision[i], lw=2, label='Class {}'.format(class_labels[i]))

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="best")
plt.title("Precision vs. Recall curve")
plt.show()

In [ ]:
# ROC Curve and AUC
from sklearn.metrics import roc_curve, auc

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label='Class {} (area = {:.2f})'.format(class_labels[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()